In [1]:
x = "stopit"

In [2]:
x[:-1]

'stopi'

In [239]:
import random
doc = "12345678901234567890123456789"
b = 10
l = random.randint(4, int(b*7/8))
print(f"b={b}\nl={l}")
doc = doc[0:l]
print(f"doc={doc}")

b=10
l=7
doc=1234567


In [266]:
lm = random.randint(1,int(l/2))
print(f"lm={lm}")
start = random.randint(2,l-lm-1)
print(f"start={start}")
prefix = doc[0:start-1]
print(f"prefix={prefix}")
masked_content = doc[start-1:start+lm-1]
print(f"masked_content={masked_content}")
suffix = doc[start+lm-1:]
print(f"suffix={suffix}\n")

lm=1
start=2
prefix=1
masked_content=2
suffix=34567


In [271]:
#rearrange: [prefix] MASK_CHAR [suffix] MASK_CHAR [masked_content] [pads]

MASK_CHAR = "\u2047" # the doublequestionmark character, for mask
PAD_CHAR = "\u25A1" # the empty square character, for pad
masked_string = prefix + MASK_CHAR + suffix + MASK_CHAR + masked_content
print(f"masked_string={masked_string}")
lp = b - len(masked_string) + 1
print(f"lp={lp}")
masked_string = masked_string + PAD_CHAR*lp
print(f"masked_string={masked_string}")
lr = len(masked_string)
print(f"lr={lr}")

masked_string=1⁇34567⁇2
lp=2
masked_string=1⁇34567⁇2□□
lr=11


In [273]:
x = masked_string[:-1]
print(f"x={x}")
y = masked_string[1:]
print(f"y={y}")

x=1⁇34567⁇2□
y=⁇34567⁇2□□


In [ ]:
import torch
x = torch.tensor([stoi[c] for c in x], dtype=torch.long)
y = torch.tensor([stoi[c] for c in y], dtype=torch.long)